In [172]:
import gurobipy as gp
from gurobipy import GRB

In [173]:
combos, spaces, profits = gp.multidict(
    {
        ("product1", "front"): [480, 3100],
        ("product2", "front"): [650, 3800],
        ("product3", "front"): [580, 3500],
        ("product4", "front"): [390, 2850],
        ("product1", "middle"): [480, 3100],
        ("product2", "middle"): [650, 3800],
        ("product3", "middle"): [580, 3500],
        ("product4", "middle"): [390, 2850],
        ("product1", "rear"): [480, 3100],
        ("product2", "rear"): [650, 3800],
        ("product3", "rear"): [580, 3500],
        ("product4", "rear"): [390, 2850],
    }
)

In [174]:
products, masses = gp.multidict(
    {
        "product1": 18,
        "product2": 15,
        "product3": 23,
        "product4": 12,
    }
)

In [175]:
positions, mass_limits, space_limits = gp.multidict(
    {"front": [10, 6800], "middle": [16, 8700], "rear": [8, 5300]}
)

In [176]:
m = gp.Model("product assign")

In [177]:
x = m.addVars(combos, name="x", vtype=GRB.CONTINUOUS)

In [178]:
m.setObjective(x.prod(profits), sense=GRB.MAXIMIZE)

In [179]:
mass_cons = m.addConstrs(
    (x.sum("*", position) <= mass_limits[position] for position in positions),
    name="mass_cons",
)

In [180]:
space_cons = m.addConstrs(
    (
        sum(x[product, position] * spaces[product, position] for product in products)
        <= space_limits[position]
        for position in positions
    ),
    name="space_cons",
)

In [181]:
product_mass_cons = m.addConstrs(
    (x.sum(product, "*") <= masses[product] for product in products),
    name="product_mass_cons",
)

In [182]:
balance_con_1 = m.addConstr(
    x.sum("*", "front") / mass_limits["front"]
    == x.sum("*", "middle") / mass_limits["middle"],
    name="balance_con_1",
)
balance_con_2 = m.addConstr(
    x.sum("*", "front") / mass_limits["front"]
    == x.sum("*", "rear") / mass_limits["rear"],
    name="balance_con_2",
)

In [183]:
m.write("assign.lp")

In [184]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 12 columns and 52 nonzeros
Model fingerprint: 0x5bdb8565
Coefficient statistics:
  Matrix range     [6e-02, 6e+02]
  Objective range  [3e+03, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 9e+03]
Presolve time: 0.00s
Presolved: 12 rows, 12 columns, 52 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2421875e+33   3.215234e+31   1.242188e+03      0s
       6    1.2151579e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.215157895e+05


In [185]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 121515.78947368421
Optimal solution:
x[product2,front] = 10.0
x[product3,middle] = 12.947368421052632
x[product4,middle] = 3.0526315789473686
x[product2,rear] = 5.0
x[product3,rear] = 3.0
